# **Digital image processing – Basics**

<div style="color:#777777;margin-top: -15px;">
<b>Author</b>: Norman Juchler |
<b>Course</b>: ADLS ISP |
<b>Version</b>: v1.2 <br><br>
<!-- Date: 30.03.2025 -->
<!-- Comments: Entirely refactored -->
</div>

In this notebook, we will learn how to handle images in Python using the OpenCV and PIL packages. 
We will also learn how to read, create and modify images using these libraries.




---

## **Preparations**

Let's begin with the usual preparatory steps...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import PIL

# Jupyter / IPython configuration:
# Automatically reload modules when modified
%load_ext autoreload
%autoreload 2

# Enable vectorized output (for nicer plots)
%config InlineBackend.figure_formats = ["svg"]

# Inline backend configuration
%matplotlib inline

# Enable this line if you want to use the interactive widgets
# It requires the ipympl package to be installed.
#%matplotlib widget

import sys
sys.path.insert(0, "../")
import isp

---

<a id='exercise1'></a>

## **&#9734;  Exercise 1 – Image representation**
We have already learned that a digital grayscale image can be represented as a 2D array of pixel intensity values. In this exercise, we will learn how to load an image and access its pixel values using the OpenCV library. OpenCV is a powerful toolkit for image processing and computer vision, particularly well-suited for real-time applications. It is written in C++ but provides convenient Python bindings.

A grayscale image is stored as a 2D array with dimensions corresponding to its height ($h$) and width ($w$), resulting in $h \times w$ intensity values. In contrast, a color image (e.g., RGB) is typically represented as a 3D array, where each pixel has three values – one for each color channel – resulting in $3 \times h \times w$ values. In Python, the most suitable data structure for this kind of data is a NumPy array.

In fact, most image processing libraries in Python either natively use NumPy arrays to represent images or provide easy ways to convert to and from NumPy arrays. This compatibility makes it convenient to combine different libraries within the same workflow.

To display an image, we can use the [`imshow()`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html) function from matplotlib.


### **Instructions:**

* Load the image "../data/images/kingfisher-gray.jpg" using the function `cv.imread()`. See [this example](https://docs.opencv.org/4.x/db/deb/tutorial_display_image.html) for reference.
* Inspect the returned array: What is its shape? What is its data type?
* Display the image using [`plt.imshow()`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html). Hint: Set the argument `cmap="gray"` for proper grayscale visualization.
* Retrieve the pixel value at position (100, 100).
* Select a region of interest (ROI) starting at pixel (75, 200) with a width of 200 and height of 100 pixels.


In [ ]:
######################
###    EXCERISE    ###
######################

# Load the image
gray = cv.imread(...)

# Print the image properties
print("Height, width:       ", ...)
print("Data type:           ", ...)
print()

# Display the image
plt.imshow(...)
plt.show()

# Read the pixel at [100,100]
print("Pixel at [100,100]:  ", ...)

# Crop the image and display it
...


---

<a id='exercise1'></a>

## **&#9734;  Exercise 2 – Digitization of images**

Just like time-based signals, digital images are represented as arrays of numbers. The digitization (or discretization) process involves two key steps:

1. **Sampling**: The continuous image is sampled at regular spatial intervals to create a grid of pixels.
2. **Quantization**: The continuous range of pixel intensities is mapped to a finite set of discrete levels.

The result is a 2D array where each value corresponds to the intensity (or color) of a pixel.


### **Instructions:**

* **Theory**: Compare sampling in time vs. space
  * Explain the key differences between sampling continuous-time signals (e.g., audio) and spatial sampling in images.
  * Consider aspects such as dimensionality, interpretation of sample points, and typical challenges in each domain.
* **Quantization**: Reducing image depth
  * What happens when you reduce the number of bits used to represent each pixel?
  * Use the image from the previous exercise.
  * Study the function quantize(image, nbits) and make sure you understand how it works.  
    Note: NumPy does not natively support arbitrary n-bit data types (e.g., 4-bit, 5-bit, etc.), so we simulate quantization using 8-bit arrays by rounding values appropriately.
  * Use the quantize() function to create lower bit-depth versions of the image and observe the visual impact.
* **Spatial sampling**: Reducing resolution
  * What happens when you reduce the number of pixels?
  * Subsample the image progressively by a factor of 2 using slicing, e.g., `gray[::2, ::2]`.
  * Reflect: Aside from lower resolution, what other effects do you expect with this type of downsampling?

In [ ]:
def quantize(image, nbits):
    """
    Method 1:
    Quantize an image to a lower bit depth. The image must be in uint8 format,
    and the number of bits must be between 1 and 8 (inclusive).
    Works for grayscale and multi-channel (e.g., RGB) images.
    """
    assert image.dtype == np.uint8, "Image must be of type uint8"
    assert 1 <= nbits <= 8, "Number of bits must be between 1 and 8"
    
    # Define bin edges to divide the 256 intensity levels
    # Example bins:
    #     nbits=8 -> [0, 1, 2, ..., 256]
    #     nbits=2 -> [0, 64, 128, 192, 256]
    #     nbits=1 -> [0, 128, 256]
    bins = np.arange(0, 256+1, 256 // 2**nbits)
    
    # Digitize returns the index of the bin each pixel belongs to. 
    # Note that the bin indices are 1-based, as the zero bin is defined
    # as (-infinity, bins[0]), which is not relevant for our case (we do not
    # have negative pixel values in uint8 images):
    #      bin 0: (-infinity, bins[0])
    #      bin 1: [bins[0], bins[1])
    #      bin 2: [bins[1], bins[2])
    #      ...
    # Hence, we can subtract 1 to start indexing from 0.
    result = np.digitize(image, bins) - 1
    
    # Scale bin indices back to intensity range (still stored as uint8)
    result *= (256 // 2**nbits)
    return result.astype(np.uint8)


def quantize(image, nbits):
    """
    Method 2:
    Quantize an image using bitwise operations for better performance.
    Equivalent to Method 1, but faster.
    """
    assert image.dtype == np.uint8, "Image must be of type uint8"
    assert 1 <= nbits <= 8, "Number of bits must be between 1 and 8"
    return ((image >> (8 - nbits)) << (8 - nbits)).astype(np.uint8)


In [ ]:
######################
###    EXERCISE    ###
######################

# Differences between temporal and spatial sampling?
# ...

# Quantization: Reduce image bit depth
nbits = [8, 6, 4, 3, 2, 1]
fig, axes = plt.subplots(2, len(nbits) // 2, figsize=(9, 5))
for i, nbit in enumerate(nbits):
    gray_quantized = ...
    ...

# Spatial subsampling: Reduce resolution
fig, axes = plt.subplots(2, len(nbits) // 2, figsize=(9, 5))
gray_sub = gray.copy()
for i in range(1,6):
    gray_sub = ...
    ...


---

<a id='exercise3'></a>

## **&#9734;  Exercise 3 – Color representation**

Gray is dull! Let’s add some color in this next exercise! We again load the same image as before, but this time in color. Let's give it a try!


In [ ]:
# Load the color image
color = cv.imread("../data/images/kingfisher.jpg")

# We use here a convenience function to display the image. 
# It embeds the image directly in the notebook.
isp.display_image(color)

# Print image dimensions: height, width, and number of color channels
print("Image shape (height, width, channels):", color.shape)

The image is indeed a color image – it's now represented using three channels. But the colors in the image look strange. This is because OpenCV loads images in **BGR format**, while Matplotlib expects **RGB format**. To correct this, you can simply reorder the channels:

In [ ]:
color_bgr = color.copy()
color_rgb = color_bgr[..., ::-1]
isp.display_image(color_rgb)

### **Instructions**:
* Examine the image's shape and depth.
* Print the pixel value at position (100, 100).
* Crop the image to the region [125:300, 300:600] and display the result.
* Convert the image to an 8-bit grayscale image using at least two different methods (hint: refer to the lecture slides).
* Use OpenCV’s built-in function [`cv.cvtColor(image, cv.COLOR_RGB2GRAY)`](https://docs.opencv.org/3.4/d8/d01/group__imgproc__color__conversions.html) for grayscale conversion.
* Optional: Apply the `quantize()` function from the previous exercise to quantize color images.

In [ ]:
######################
###    EXERCISE    ###
######################

print("Image shape (height, width, channels):", ...)
print("Data type:                            ", ...)
print("Pixel value at [100, 100]:            ", ...)
print()

# Crop the image
color_cropped = ...
isp.display_image(color_cropped, scale=2.0)

# Convert to grayscale:
gray_method1 = ...
gray_method2 = ...
gray_opencv = ...
isp.show_image_grid((color_rgb, gray_method1, gray_method2, gray_opencv),
                    titles=("Original", "Method 1", "Method 2", "OpenCV"))

# Convert to n-bit image
nbits = [8, 6, 4, 3, 2, 1]
fig, axes = plt.subplots(2, len(nbits)//2, figsize=(12, 6))
for i, nbit in enumerate(nbits):
    color_quantized = ...
    axes[i//3, i%3].imshow(color_quantized, cmap="gray")
    axes[i//3, i%3].set_title(f"{nbit}-bit (2^{3*nbit} colors)")
plt.show()


### **Digression: Pillow, color bands an dithering**


[Pillow (PIL)](https://pillow.readthedocs.io/en/stable/) is a powerful and user-friendly alternative to OpenCV for image processing. It offers a simpler API and includes several advanced features. In this course, we will use Pillow here and there for tasks that are more easily or effectively handled with its tools.

One such feature is its built-in color quantization method: [`PIL.Image.quantize()`](https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.quantize). It allows us to easily reduce the number of colors in an image, and optionally apply dithering to improve visual quality.

**Dithering** is a technique that reduces visible quantization artifacts (such as color banding) by intentionally adding structured noise to the image. This noise is often correlated with the quantization error, which helps spread the error more evenly across the image. The result is a smoother, more natural-looking image.

Below, we compare quantized images with and without dithering. Notice the visible color bands in the non-dithered version. The dithered image appears more natural, not only due to the dithering itself, but also because Pillow uses a more sophisticated quantization algorithm (e.g. k-means clustering), as opposed to the simple binning method we used earlier.

In [ ]:
# Convert the image (represented as NumPy array) to a Pillow image. Easy!
color_pil = PIL.Image.fromarray(color_rgb)

#############################
# Quantize WITHOUT dithering
#############################
ncolors = [256, 128, 64, 32, 16, 8]
images = []
for i, ncols in enumerate(ncolors):
    img = color_pil.quantize(colors=ncols,
                             method=PIL.Image.Quantize.MAXCOVERAGE,
                             kmeans=1, )
    images.append(img.convert("RGB"))

# Display results. Use a convenience function to show the images in a grid.
isp.show_image_grid(images, titles=[f"{nc} colors" for nc in ncolors],
                    figsize=(9, 6), ncols=3,
                    header="Quantize WITHOUT dithering")


#############################
# Quantize WITH dithering
#############################

# Dithering only works with paletted images, where each pixel holds an
# index into a limited color palette. Pillow can generate a palette 
# adaptively or use the fixed "web-safe" palette of 256 colors.
ncolors = [256, 128, 64, 32, 16, 8]
images = []
for i, ncols in enumerate(ncolors):
    # Convert to a paletted image.
    img = color_pil.convert("P", dither=None, palette=PIL.Image.Palette.ADAPTIVE)
    #img = color_pil.convert("P", dither=None, palette=PIL.Image.Palette.WEB)
    
    # Apply quantization with Floyd–Steinberg dithering
    img = img.quantize(colors=ncols, 
                       method=PIL.Image.Quantize.MAXCOVERAGE, 
                       kmeans=1, 
                       dither=PIL.Image.Dither.FLOYDSTEINBERG)
    images.append(img.convert("RGB"))

isp.show_image_grid(images, titles=[f"{nc} colors" for nc in ncolors],
                    figsize = (9, 6),
                    header="Quantize WITH dithering")

## **&#9734;&#9734; More on color spaces**

**Comment 1**: Some color spaces are more perceptually uniform than RGB. For example, the [CIE Lab](https://en.wikipedia.org/wiki/CIELAB_color_space) color space is specifically designed so that small changes in its values correspond to similarly small changes in perceived color. In contrast, in the RGB space, Euclidean distances between colors do not reliably reflect how different those colors appear to the human eye. Nevertheless, for display purposes, we often need to convert images back to RGB.


**Comment 2**: Several color spaces separate intensity (or *luma*) from color (or *chroma*) information. This distinction is useful because the human visual system is much more sensitive to variations in brightness than to changes in color. Examples of such color spaces include [Y'UV](https://en.wikipedia.org/wiki/Y%E2%80%B2UV), [YCbCr](https://en.wikipedia.org/wiki/YCbCr), [HSL (or HSV)](https://en.wikipedia.org/wiki/HSL_and_HSV), and [CIE Lab](https://en.wikipedia.org/wiki/CIELAB_color_space). In many image processing applications, it is therefore common to quantize the intensity channel with more bits than the chroma channels.

In the following, we will construct a color wheel using one of these color spaces, ensuring that all colors have a uniform intensity level (uniform *luma* L). Since the image will eventually be displayed in RGB, we will convert the color wheel back to the RGB space for visualization.

Now, what should we expect if we convert this RGB color wheel to a grayscale image? Since the intensity (L) values were uniform in the Lab space, the grayscale version should appear relatively uniform as well. Let's check if that holds true. Any slight variations in gray levels may be due to inaccuracies or approximations in the color conversion functions between Lab and RGB.

For information on color space conversions, see [here](https://docs.opencv.org/4.x/de/d25/imgproc_color_conversions.html).

In [ ]:
# def create_color_wheel1(size=1000, rel_width=0.15, intensity=128):
#     """
#     Create a color wheel with uniform intensity levels. (Slow version.)
#     """
#     # Create a color wheel with uniform intensity levels
#     img = np.zeros((size, size, 3), dtype=np.uint8)
#     width = rel_width*size
#     L = intensity
#     for i in range(size):
#         for j in range(size):
#             # Compute the angle and radius
#             x = i - size // 2
#             y = j - size // 2
#             angle = np.arctan2(y, x)
#             radius = np.sqrt(x**2 + y**2)
#             # Compute the chroma values (between 0 and 255)
#             a = (np.cos(angle)+1) * 128
#             b = (np.sin(angle)+1) * 128
#             # Fix some rounding error by offsetting a and b
#             a = a - 0.5
#             b = b - 0.5
#             # Convert to uint8
#             a = a.astype(np.uint8)
#             b = b.astype(np.uint8)
#             # On circle?
#             mask = (radius > size//2-width) & (radius < size//2)
#             img[i, j] = [L, a, b] if mask else [0, 128, 128]
#     return img

def create_color_wheel(size=1000, rel_width=0.15, intensity=128):
    """
    This version is much faster than the previous one.
    """
    L = intensity
    width = rel_width*size
    J, I = np.meshgrid(np.arange(size), np.arange(size))
    X = I - size // 2
    Y = J - size // 2
    angle = np.arctan2(Y, X)
    radius = np.sqrt(X**2 + Y**2)
    L = np.ones_like(X, dtype=np.uint8) * L
    a = ((np.cos(angle)+1) * 128 - 0.5).astype(np.uint8)
    b = ((np.sin(angle)+1) * 128 - 0.5).astype(np.uint8)
    img = np.stack((L, a, b), axis=-1)
    mask = (radius > size//2-width) & (radius < size//2)
    # Black in LAB space
    img[~mask] = [0, 128, 128]
    return img


# Create the color wheel. The first channel is the luminance, the other two
# channels define the chrominance.
target_intensity = 128
img_lxx = create_color_wheel(size=1000, rel_width=0.15, 
                             intensity=target_intensity)

# Convert to RGB (for display)
# All of the following color spaces have in the first channel the luminance
# and in the other two channels the chrominance. We can construct the 
# color wheel in the same way for all of them.
#   YCrCb, Yuv, Luv, Lab

images = []
titles = []

print("Target: Intensity:", target_intensity)

img_rgb = cv.cvtColor(img_lxx, cv.COLOR_YUV2RGB)
img_gray = cv.cvtColor(img_rgb, cv.COLOR_RGB2GRAY)
images.append(img_rgb)
images.append(img_gray)
titles.append("YUV color circle")
titles.append("YUV intensity")
print("YUV:    Gray level range", img_gray[img_gray>0].min(), img_gray.max())

img_rgb = cv.cvtColor(img_lxx, cv.COLOR_YCrCb2RGB)
img_gray = cv.cvtColor(img_rgb, cv.COLOR_RGB2GRAY)
images.append(img_rgb)
images.append(img_gray)
titles.append("YCrCb color circle")
titles.append("YCrCb intensity")
print("YCrCb:  Gray level range", img_gray[img_gray>0].min(), img_gray.max())

img_rgb = cv.cvtColor(img_lxx, cv.COLOR_LUV2RGB)
img_gray = cv.cvtColor(img_rgb, cv.COLOR_RGB2GRAY)
images.append(img_rgb)
images.append(img_gray)
titles.append("LUV color circle")
titles.append("LUV intensity")
print("LUV:    Gray level range", img_gray[img_gray>0].min(), img_gray.max())

img_rgb = cv.cvtColor(img_lxx, cv.COLOR_LAB2RGB)
img_gray = cv.cvtColor(img_rgb, cv.COLOR_RGB2GRAY)
images.append(img_rgb)
images.append(img_gray)
titles.append("LAB color circle")
titles.append("LAB intensity")
print("LAB:    Gray level range", img_gray[img_gray>0].min(), img_gray.max())

isp.show_image_grid(images, titles=titles,
                    figsize=(6, 12), ncols=2,
                    suppress_info=True,
                    header="Color wheel in different color spaces")


---

<a id='exercise4'></a>

## **&#9734;  Exercise 4 – Construct an image**

In this exercise, you will create an image from scratch. Your task is to generate a checkerboard pattern consisting of 8×8 squares, where each square measures 32×32 pixels. The squares should alternate between two colors of your choice (e.g., red and green or black and white). The final image should be 256×256 pixels in total.

Complete the function template below to implement your solution.

In [ ]:
######################
###    EXERCISE    ###
######################

def create_checkerboard(square_size=32, checker_size=8, color1=[0,0,0], color2=[255,255,255]):
    """Create a checkerboard image with the specified square size and checker size."""
    image_size = square_size * checker_size
    # Complete this function
    iamge = ...
    return image
    
checker = create_checkerboard(square_size=32, 
                              checker_size=8, 
                              color1=[255,188,188], 
                              color2=[255,255,188]) 
isp.display_image(checker, scale=1.5)

---

<a id='exercise5'></a>

## **&#9734;  Exercise 5 – Gamma correction**

Gamma correction is a non-linear operation used to adjust the brightness of an image. It compensates for the non-linear response of the human eye to light intensity, as well as the non-linear behavior of certain display devices.

The gamma correction formula is:

$$I_{out} = I_{in} ^ {\;\gamma}$$

Here, $\gamma > 0$ is the correction factor:
* For $\gamma=1$, the image remains unchanged.
* For $\gamma>1$, the image appears darker.
* For $\gamma<1$, the image appears brighter.


**Hint**: Gamma correction should be applied to images in float format, with pixel values normalized to the range [0, 1]. This means you will need to convert the image's data type and scale the values accordingly. (Note: Matplotlib can display floating-point images just like images with integer pixel values.)


### **Instruction:**
Complete the function template below to apply gamma correction.

In [ ]:
######################
###    EXERCISE    ###
######################

# Solution
def gamma_correction(image, gamma):
    """Apply gamma correction to an image with the specified gamma value."""
    # Conver to float if the image is in uint8 format
    image = ...
    # Apply the gamma correction
    corrected = ...
    return corrected

# Test the function.
image = color_rgb.copy()
gamma = 1.5
corrected = gamma_correction(image, gamma)
isp.show_image_grid((image, corrected), titles=("Original", f"Gamma={gamma}"))

---

<a id='exercise6'></a>

## **&#9734;  Exercise 6 – Color and intensity histograms**

### **Instructions:**
* Carefully read the following section
* Make sure you understand the concepts of intensity and color histograms
* Familiarize yourself with the types of color transformations explored here

<br>

The *distribution of intensity* values in an image (or a specific channel) characterizes an image. We can use image *histogram* to show how many pixels fall into each intensity level. Histograms are useful to analyze an image's contrast, brightness, and dynamic range.


In [ ]:
# Compute the normalized histogram and cumulative distribution function (CDF)
hist, bins = np.histogram(gray.flatten(), bins=256, range=[0,256], density=True)
cdf = hist.cumsum()
hist /= hist.max()  # Normalize histogram for display purposes

# Visualize the histogram and CDF
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(gray, cmap="gray")
axes[0].axis("off")
axes[1].plot(hist, label="Histogram (normalized)")
axes[1].plot(cdf, label="CDF")
axes[1].set_xlabel("Pixel value")
axes[1].set_ylabel("Density")
axes[1].grid(axis="y")
axes[1].legend()

fig.tight_layout()
plt.show()

For this input image, we observe that most pixel values are concentrated between 120 and 180, with very few pixels below 100. This distribution indicates that the image has a limited dynamic range in terms of brightness levels.

The histogram plot above also shows the cumulative distribution function (CDF) of the pixel values. The CDF represents the fraction of pixels with intensity values less than or equal to a given value. For example, a CDF value of less than 0.1 at intensity 100 means that fewer than 10% of the pixels have values below 100.


Such a distribution typically corresponds to a low-contrast image, where pixel values are clustered within a narrow range. To **enhance contrast**, we can apply contrast stretching, which remaps the pixel values to span a broader range. One common technique for this is **histogram equalization** – a method that improves image contrast by redistributing pixel values to achieve a more uniform histogram. The aim is to spread out the intensities so that all values occur more evenly, enhancing the visual quality of the image.

In [ ]:
# Apply histogram equalization
gray = cv.imread("../data/images/kingfisher-gray.jpg", cv.IMREAD_GRAYSCALE)
gray_equalized = cv.equalizeHist(gray)
hist_equalized, bins = np.histogram(gray_equalized.flatten(), 
                                    bins=256, range=[0,256], 
                                    density=True)
cdf_equalized = hist_equalized.cumsum()
hist_equalized /= hist_equalized.max()
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(gray_equalized, cmap="gray")
axes[0].axis("off")
axes[1].plot(hist_equalized, label="Histogram (normalized)")
axes[1].plot(cdf_equalized, label="CDF")
axes[1].set_xlabel("Pixel value")
axes[1].legend()
fig.tight_layout()

The equalized image shows a more uniform histogram, meaning the pixel values are more evenly distributed across the intensity range. The CDF appears now linear, which further indicates an improvement in image contrast. However, histogram equalization can also amplify noise or unwanted artifacts. For example, quantization bands that were subtle in the original image may become more pronounced after equalization.

For **color images**, we can apply histogram equalization to each channel independently. However, this can distort colors. A better approach is to convert the image to a color space that separates brightness from color information—such as YUV or HSL.

In [ ]:
# Let's define a function to visualize the histogram of all 3 channels in the image
def visualize_histogram(image, title="Histogram", channel_names="RGB"):

    def visualize_channel(img, title, ax, color):
        hist, bins = np.histogram(img.flatten(), bins=256, range=[0,256], density=True)
        cdf = hist.cumsum()
        hist /= hist.max()
        ax.plot(hist, label="Histogram (normalized)", color=color)
        ax.plot(cdf, label="CDF", color="k", linestyle=":")
        if False:
            ax.set_xlabel("Pixel value")
            ax.legend()
        ax.set_title(title)

    """Visualize the histogram of an image."""
    if image.ndim == 1:
        return visualize_channel(image, title, plt.gca())
    else:
        nchannels = image.shape[-1]
        fig, axes = plt.subplots(1, nchannels+1, figsize=(9, 2))
        axes[0].imshow(image, cmap="gray" if nchannels == 1 else None)
        axes[0].axis("off")
        axes[0].set_title(title)
        axes[0].set_anchor("N")
        titles = ["%s channel" % name for name in channel_names]
        for i in range(image.shape[-1]):
            visualize_channel(image[..., i], 
                              title=titles[i],
                              ax=axes[i+1], color=isp.PALETTE_RGB[i])
    #fig.suptitle(title)
    plt.tight_layout()


# Convert the image to float
color_equalized = np.stack([cv.equalizeHist(color_bgr[...,i]) for i in range(3)], axis=2)
visualize_histogram(color_rgb, title="Original")
visualize_histogram(color_equalized, title="RGB equalized")
# Better, convert to HSL color space and apply histogram equalization 
# to the L (luminance) channel only!
color_hls = cv.cvtColor(color_rgb, cv.COLOR_RGB2HLS)
color_hls[..., 1] = cv.equalizeHist(color_hls[..., 1])
color_equalized = cv.cvtColor(color_hls, cv.COLOR_HLS2RGB)
visualize_histogram(color_equalized, 
                    title="HLS equalized",
                    channel_names=["H", "L*", "S"])
# Also the YUV color space can be used
color_hls = cv.cvtColor(color_rgb, cv.COLOR_RGB2YUV)
color_hls[..., 0] = cv.equalizeHist(color_hls[..., 0])
color_equalized = cv.cvtColor(color_hls, cv.COLOR_YUV2RGB)
visualize_histogram(color_equalized, 
                    title="YUV equalized", 
                    channel_names=["Y*", "U", "V"])


**Observations:** Histogram equalization can also be applied to color images. However, applying it independently to each RGB channel often leads to unnatural color distortions. A better approach is to apply the method to a single channel that represents the image’s luminance or intensity. This can be done by converting the image to a color space such as HLS or YUV, and applying equalization to the luminance channel only. This typically produces more visually appealing results.

Overall, histogram equalization is a simple and effective technique for enhancing image contrast.

Another related method is **histogram matching** (also known as histogram specification). Instead of flattening the histogram, it adjusts the contrast of an image by matching its histogram to that of a reference image. In Python, this can be done using the [`skimage.exposure.match_histograms()`](https://scikit-image.org/docs/stable/api/skimage.exposure.html=) function from the scikit-image library. It takes an input image and a reference image, and returns a version of the input image whose histogram closely resembles that of the reference.

In [ ]:
#from skimage import exposure
from skimage.exposure import match_histograms

color1 = color_rgb.copy()
color2 = cv.imread("../data/images/veggies.jpg")
color2 = cv.cvtColor(color2, cv.COLOR_BGR2RGB)
color3 = cv.imread("../data/images/flowers.jpg")
color3 = cv.cvtColor(color3, cv.COLOR_BGR2RGB)

matched12 = match_histograms(color1, color2, channel_axis=-1)
matched23 = match_histograms(color1, color3, channel_axis=-1)

isp.show_image_grid((color1, color2, matched12), 
                    titles=("Input", "Reference", "Matched"),
                    suppress_info=True,
                    figsize=(9, 6))
isp.show_image_grid((color1, color3, matched23), 
                    titles=("Input", "Reference", "Matched"),
                    suppress_info=True,
                    figsize=(9, 6))

visualize_histogram(color_rgb, title="Input")
visualize_histogram(color3, title="Input")
visualize_histogram(matched23, title="Matched")

Notice how the CDFs of the matched image closely resemble those of the reference image – demonstrating the effectiveness of histogram matching.

Histograms are also a useful tool for understanding the effects of various color and **intensity adjustments** on an image. Below, we demonstrate how different operations impact the image and its histogram:

* No operation (copy image)
* Inversion
* Brightness increase / decrease
* Contrast stretching / compression
* Clipping (limiting values to a specified [min, max] range)
* Binarization (thresholding)
* Ggamma correction


In [ ]:
labels = []
lookup_tables = []
image = gray.copy()
results = []
normalize = False  # Do not normalize the values for display!

labels.append("Identity")
lookup_table = np.arange(256)
lookup_tables.append(lookup_table)

labels.append("Inverted")
lookup_table = 255 - np.arange(256)
lookup_tables.append(lookup_table)

labels.append("Increase brightness")
lookup_table = np.clip(np.arange(256) + 50, 0, 255)
lookup_tables.append(lookup_table)

labels.append("Decrease brightness")
lookup_table = np.clip(np.arange(256) - 50, 0, 255)
lookup_tables.append(lookup_table)

labels.append("Contrast stretch")
# Lookup such that mean remains equal, but the range is stretched
# (The peak of the histogram is at 163)
center = 163
lookup_table = np.clip((np.arange(256) - center) * 3 + center, 0, 255)
lookup_tables.append(lookup_table)

labels.append("Contrast squeeze")
lookup_table = np.clip((np.arange(256) - center) / 6 + center, 0, 255)
lookup_tables.append(lookup_table)

labels.append("Clipping")
lookup_table = np.clip(np.arange(256), 135, 175)
lookup_tables.append(lookup_table)

labels.append("Threshold")
lookup_table = (np.arange(256) > 128) * 255
lookup_tables.append(lookup_table)

labels.append("Gamma=0.5")
lookup_table = (np.arange(256) / 255) ** 0.5 * 255
lookup_tables.append(lookup_table)

labels.append("Gamma=2.0")
lookup_table = (np.arange(256) / 255) ** 2.0 * 255
lookup_tables.append(lookup_table)


# Visualize the results
for label, lookup in zip(labels, lookup_tables):
    result = lookup[image]
    fig, axes = plt.subplots(1, 3, figsize=(9, 3))
    # Display the result
    if not normalize:
        vmin, vmax = 0, 255
    else:
        vmin, vmax = result.min(), result.max()
    axes[0].imshow(result, cmap="gray", vmin=vmin, vmax=vmax)
    axes[0].set_title(label, fontweight="bold")
    axes[0].axis("off")
    axes[0].set_anchor("N")
    # Display lookup table
    axes[1].plot(lookup)
    axes[1].set_title("Lookup table")
    axes[1].set_xlabel("Input value")
    axes[1].set_ylabel("Output value")
    axes[1].set_xlim([0-5, 255+5])
    axes[1].set_ylim([0-5, 255+5])
    axes[1].set_aspect("equal")
    axes[1].grid(axis="y")
    # Display the histograms (before, after)
    hist, bins = np.histogram(image.flatten(), bins=256, range=[0,256], density=True)
    axes[2].plot(hist, label="Before")
    hist, bins = np.histogram(result.flatten(), bins=256, range=[0,256], density=True)
    axes[2].plot(hist, label="After")
    axes[2].set_title("Histogram")
    axes[2].legend()
    axes[2].grid(axis="y")
    fig.tight_layout()